In [3]:
!pip install cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.6/594.6 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [4]:
import cirq
import tensorflow as tf
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [5]:
# Get the path of files
img_dir='/content/drive/MyDrive/Colab Notebooks/brain-tumor-detection/'
no_images=os.listdir(img_dir + 'no/')
yes_images=os.listdir(img_dir + 'yes/')

In [6]:

# Initialize dataset and label arrays
dataset=[]
label=[]

IMG_SIZE = 128

In [7]:
# For images labelled with no
for image_name in no_images:
  if(image_name.split('.')[1]=='jpg'):
    image=cv2.imread(img_dir + 'no/' +image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((IMG_SIZE, IMG_SIZE))
    dataset.append(np.array(image))
    label.append(0)

In [8]:
# For images labelled with yes
for image_name in yes_images:
  if(image_name.split('.')[1]=='jpg'):
    image=cv2.imread(img_dir + 'yes/' +image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((IMG_SIZE, IMG_SIZE))
    dataset.append(np.array(image))
    label.append(1)

In [9]:
# Convert dataset and label to numpy arrays  
dataset=np.array(dataset)
label=np.array(label)

In [10]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=42)



In [11]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Flatten the images into a vector of pixel values
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Convert the labels to binary values
y_train = (y_train == 0).astype(int)
y_test = (y_test == 0).astype(int)

In [12]:
# Define a quantum circuit
def quantum_circuit(qubits, X):
    circuit = cirq.Circuit()
    # Encode the input features as amplitudes on the qubits
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] > 0:
                circuit.append(cirq.X(qubits[i]))
        circuit.append(cirq.H(qubits[i]))
    # Add a CNOT gate between each pair of qubits
    for i in range(len(qubits)-1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i+1]))
    # Measure the last qubit
    circuit.append(cirq.measure(qubits[-1], key='m'))
    return circuit


In [13]:
# Define the classical neural network
def classical_network(X_train, y_train, X_val, y_val):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test))
    y_pred = model.predict(X_test).flatten()
    return (y_pred > 0.5).astype(int)

In [ ]:
# Run the quantum and classical models
qubits = cirq.GridQubit.rect(1, 784)
quantum_model = cirq.Simulator().run(quantum_circuit(qubits, X_test), repetitions=0)
y_pred_quantum = np.array([np.mean(measurement['m']) for measurement in quantum_model.measurements])
y_pred_classical = classical_network(X_train, y_train, X_test, y_test)

In [ ]:
# Print the accuracy of both models
print("Quantum model accuracy: ", np.mean(y_pred_quantum == y_test))
print("Classical model accuracy: ", np.mean(y_pred_classical == y_test))